# Prompt Engineering Illustration

A common real world use case is to recognize the presence of a phrase in text. For example, in the manufacturing context, there may be text descriptions to describe the suspected cause for a product failure. Analyzing such data can be useful for the manufacturer to identify common causes of failures and take steps to address these common causes. For more sophisiticated applications, data related to the products can be used to correlate to these causes of failure and machine learning models can be employed to predict these causes.

However, the main challenge is identifying key descriptions in the data. This notebook gives a high level illustration of how we can coax LLMs to identify key descriptions in the data. While this is only an example, these snippet of codes can be extended to for more sophisticated applications.

In [1]:
import time, torch

from langchain.chains import LLMChain
from langchain.llms import LlamaCpp
from langchain.prompts import PromptTemplate

In [2]:
# Check cuda is available.
device = "cuda" if torch.cuda.is_available() else "cpu"
print("Device used for LLM: %s" % device)

Device used for LLM: cuda


###
Note that apart from experimenting with different model parameters, different LLM models might also perform differently for different use cases. Hence, do try different models and see how they perform.

In [27]:
####################################################################################################
####################################################################################################
# Model Setup
####################################################################################################
####################################################################################################
# Various models in path to choose from:
# "f:/llm_models/vicuna-13b-v1.5.Q5_K_M.gguf", 
# "f:/llm_models/llama-2-13b-chat.Q5_K_M.gguf", 
# "f:/llm_models/llama-2-7b.Q6_K.gguf",
# "f:/llm_models/llama-2-7b.Q4_K_M.gguf", 
# "f:/llm_models/wizardlm-13b-v1.2.Q5_K_M.gguf"

# Chosen model for the task.
path_model = "f:/llm_models/llama-2-13b-chat.Q5_K_M.gguf"



# Load the model and define the parameters
# It appears that non-Llama models could also use the same class for executing commands.
llm = LlamaCpp(model_path = path_model,
                n_ctx = 500,           # Size of the input context. The larger the slower the processing time.
                n_gpu_layers = 24,     # number of gpu layers. to set as high as possible to offload work from cpu to gpu
                n_batch = 512,         # batches for processing
                n_threads = 20,        # cpu threads, set as high as possible as well.
                top_k = 20,            # consider top_k possible next words
                top_p=1,
                temperature = 0.01,
                repeat_penalty=1.5,
                echo=True,
                max_tokens = 10,       # max tokens allowed to be returned.
                verbose = True
    )


AVX = 1 | AVX2 = 1 | AVX512 = 0 | AVX512_VBMI = 0 | AVX512_VNNI = 0 | FMA = 1 | NEON = 0 | ARM_FMA = 0 | F16C = 1 | FP16_VA = 0 | WASM_SIMD = 0 | BLAS = 1 | SSE3 = 1 | SSSE3 = 0 | VSX = 0 | 


## Prompt 1

Prompts can play a big role in whether or not you can get the LLM to behave as you would like. Following is an example of a prompt which works in the most straightforward examples.

In [39]:
####################################################################################################
####################################################################################################
# Prompt Template and chain intialization
####################################################################################################
####################################################################################################
prompt_template='''SYSTEM: You are a helpful, obedient and honest assistant. Always answer as helpfully.

USER: '{prompt_input}'

ASSISTANT:
'''

prompt = PromptTemplate(template = prompt_template, input_variables = ["prompt_input"])
chain = LLMChain(llm = llm, prompt = prompt, verbose = True)

In [40]:
# A use case specific template for user to easily key in a sentence to detect a particular phrase
def user_prompt_template(sentence):
    output = "Help me check if the following sentence contains any word or phrase that describes 'Singapore'. " + \
                "Note that Singapore can be written as 'Sing', Singapura or or something similar. " + \
                    "The spelling might also be wrong or the description might differ slightly but they mean the same thing. " + \
                        "Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: '" + sentence + "'"
    return output

**Example 1**: Unsuccessful example.

In [41]:
# the sentence which we want to evaluate
response = chain.run(prompt_input = user_prompt_template("We flew SQ flight to Malaysia"))

response



> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful, obedient and honest assistant. Always answer as helpfully.

USER: 'Help me check if the following sentence contains any word or phrase that describes 'Singapore'. Note that Singapore can be written as 'Sing', Singapura or or something similar. The spelling might also be wrong or the description might differ slightly but they mean the same thing. Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: 'We flew SQ flight to Malaysia''

ASSISTANT:



Llama.generate: prefix-match hit



> Finished chain.


'No'

**Example 2**: Successful example.

In [45]:
# the sentence which we want to evaluate
response = chain.run(prompt_input = user_prompt_template("We flew Singapore Airlines flight to Malaysia"))

response



> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful, obedient and honest assistant. Your main task is to identify the presence of words in a sentence. Some times the word might be mis-spelt or another word could be used to describe the same thing. Excercise some judgement when making these identifications.


USER: 'Help me check if the following sentence contains any word or phrase that describes 'Singapore'. Note that Singapore can be written as 'Sing', Singapura or or something similar. The spelling might also be wrong or the description might differ slightly but they mean the same thing. Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: 'We flew Singapore Airlines flight to Malaysia''

ASSISTANT:



Llama.generate: prefix-match hit



> Finished chain.


'Yes'

## Prompt 2

As noted, despite the user inputs to note that there maybe other ways of describing Singapore, the LLM was not able to realize that the text SQ relates the Singapore Airlines and returned a No response. Following is an example of a prompt which is able to overcome the previous limitation.

In [42]:
prompt_template='''SYSTEM: You are a helpful, obedient and honest assistant. Your main task is to identify the presence of words in a sentence. Some times the word might be mis-spelt or another word could be used to describe the same thing. Excercise some judgement when making these identifications.


USER: '{prompt_input}'

ASSISTANT:
'''
prompt = PromptTemplate(template = prompt_template, input_variables = ["prompt_input"])
chain = LLMChain(llm = llm, prompt = prompt, verbose = True)

**Example 1**: Success

In [43]:
# the sentence which we want to evaluate
response = chain.run(prompt_input = user_prompt_template("We flew SQ flight to Malaysia"))

response



> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful, obedient and honest assistant. Your main task is to identify the presence of words in a sentence. Some times the word might be mis-spelt or another word could be used to describe the same thing. Excercise some judgement when making these identifications.


USER: 'Help me check if the following sentence contains any word or phrase that describes 'Singapore'. Note that Singapore can be written as 'Sing', Singapura or or something similar. The spelling might also be wrong or the description might differ slightly but they mean the same thing. Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: 'We flew SQ flight to Malaysia''

ASSISTANT:



Llama.generate: prefix-match hit



> Finished chain.


'Yes'

**Example 2**: Success

In [47]:
# the sentence which we want to evaluate
response = chain.run(prompt_input = user_prompt_template("We flew Singapore Airlines flight to Malaysia"))

response



> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful, obedient and honest assistant. Your main task is to identify the presence of words in a sentence. Some times the word might be mis-spelt or another word could be used to describe the same thing. Excercise some judgement when making these identifications.


USER: 'Help me check if the following sentence contains any word or phrase that describes 'Singapore'. Note that Singapore can be written as 'Sing', Singapura or or something similar. The spelling might also be wrong or the description might differ slightly but they mean the same thing. Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: 'We flew Singapore Airlines flight to Malaysia''

ASSISTANT:



Llama.generate: prefix-match hit



> Finished chain.


'Yes'

**Example 3**: Success

In [46]:
# the sentence which we want to evaluate
response = chain.run(prompt_input = user_prompt_template("We are the champions of the world."))

response



> Entering new LLMChain chain...
Prompt after formatting:
SYSTEM: You are a helpful, obedient and honest assistant. Your main task is to identify the presence of words in a sentence. Some times the word might be mis-spelt or another word could be used to describe the same thing. Excercise some judgement when making these identifications.


USER: 'Help me check if the following sentence contains any word or phrase that describes 'Singapore'. Note that Singapore can be written as 'Sing', Singapura or or something similar. The spelling might also be wrong or the description might differ slightly but they mean the same thing. Upon detecting the phrase, answer only Yes. Otherwise Answer No. The sentence is: 'We are the champions of the world.''

ASSISTANT:



Llama.generate: prefix-match hit



> Finished chain.


'No'